In [1]:
import pandas as pd
import numpy as np
import os
import re
import time

import requests
from bs4 import BeautifulSoup

from scrap_utils import Quote, convert_manual_author, extrat_quote, tokenize, get_babelio_citation, scrap_recent_babelio
%load_ext autoreload
%autoreload 2



def extract_poems_eclair(url = "https://www.eternels-eclairs.fr/poemes-prevert.php"):
    response = requests.get(url)
    html_content = response.content # .decode('utf-8')
    soup = BeautifulSoup(html_content, 'html.parser')

    poems = []
    for ii, card in enumerate(soup.find_all('div', class_='card')):
        try:
            # (text, title, book, auteur))
            if card.find('div', class_='poeme-texte') is not None:
                text = '\n'.join([str(x) for x in card.find('div', class_='poeme-texte').p.contents]).replace('\n<br/>\n', '\n').replace('<br/>', '').strip()
            elif card.find('p', class_='respecter-espaces-bruts') is not None:
                text = card.find('p', class_='respecter-espaces-bruts').text.strip()
            else:
                text = card.find('div', class_='poeme-texte haiku').text.strip()
            try:
                title = card.h3.text.strip()
            except:
                title = None
            try:
                book = card.find('figcaption').cite.text.strip() # book
            except:
                book = None
            auteur = card.find('figcaption').text.split("\n")[1].strip()[2:]
            poems.append(Quote(text, author = auteur, book = book,
                               source = 'eternels-eclairs', title=title,
                               url = url))
        except Exception as e:
            try:
                text = card.p.text
                title = card.h3.text
                figure_element = card.find('figure')
                figcaption_element = figure_element.find('figcaption')
                author_and_book = figcaption_element.text.strip()
                author, book = author_and_book.split(',', 1)
                author = author.strip()[2:]
                book = book.strip()
                poems.append(Quote(text, author = author, book = book,
                               source = 'eternels-eclairs', title=title,
                               url = url))
            except Exception as e:
                if ii > 0:
                    print(ii, "Error:", e) #, e.__class__, e.__class__.__name__)
                    # print(card)
    return poems


In [ ]:
# !python -m pip install pyperclip

# # import streamlit.components.v1 as components
# from streamlit_extras.let_it_rain import rain
# # import streamlit_antd_components as sac
# from st_keyup import st_keyup
# import streamlit_wordcloud as wordcloud

# pip install streamlit-wordcloud
# pip install extra-streamlit-components
# pip install streamlit-keyup
# pip install streamlit-extras

: 

In [2]:
!python3 -m pip install pyperclip

In [ ]:

l_url = """https://www.eternels-eclairs.fr/poemes-prevert.php
https://www.eternels-eclairs.fr/poemes-chansons-textes-paroles-jacques-brel.php
https://www.eternels-eclairs.fr/poemes-victor-hugo.php
https://www.eternels-eclairs.fr/poemes-jean-de-la-fontaine.php
https://www.eternels-eclairs.fr/poemes-rimbaud.php
https://www.eternels-eclairs.fr/poemes-rilke.php
https://www.eternels-eclairs.fr/poemes-apollinaire.php
https://www.eternels-eclairs.fr/poemes-eluard.php
https://www.eternels-eclairs.fr/poemes-maurice-careme.php
https://www.eternels-eclairs.fr/poemes-baudelaire.php
https://www.eternels-eclairs.fr/poemes-verlaine.php
https://www.eternels-eclairs.fr/anthologie-haikus-livres.php
https://www.eternels-eclairs.fr/haikus-japonais.php
https://www.eternels-eclairs.fr/haikus-basho-buson-issa-shiki-santoka.php
https://www.eternels-eclairs.fr/poesie-poemes-beaux-sonnets.php
https://www.eternels-eclairs.fr/poesie-celebre-les-plus-beaux-poemes-et-poemes-les-plus-connus.php"""

l_url = l_url.split('\n')
# l_url = ["https://www.eternels-eclairs.fr/poemes-verlaine.php",]

for url in l_url:
    print(url)
    result = extract_poems_eclair(url)
    print("\t--- nb quotes:", len(result))
    # print(result[0])
    if len(result) > 0:
        df_quote = pd.DataFrame([quote.__dict__ for quote in result])
        df_quote.to_csv(f'data/eclair/{url[url.index(".fr/") + 4:][:-4]}.csv')
    time.sleep(10)

In [115]:
# + sonnet best_poemes (remove double \n)
# + haiku (remove long spaces)

babe = os.listdir("data/eclair")
ldf = []
for file in babe:
    df = pd.read_csv(f"data/eclair/{file}")
    if "haiku" in file:
        df.haiku = True
    else:
        df.haiku = False
    print(file, df.shape)
    ldf.append(df)

df = pd.concat(ldf)
print('Done.')
print(' --- ', df.shape)
df.to_csv('data/eclair_data.csv')
print('Done.')



poesie-celebre-les-plus-beaux-poemes-et-poemes-les-plus-connus.csv (10, 15)
poemes-chansons-textes-paroles-jacques-brel.csv (13, 15)
poemes-victor-hugo.csv (16, 15)
poesie-poemes-beaux-sonnets.csv (16, 15)
poemes-rilke.csv (17, 15)
poemes-apollinaire.csv (10, 15)
poemes-prevert.csv (13, 15)
poemes-eluard.csv (11, 15)
poemes-maurice-careme.csv (16, 15)
poemes-baudelaire.csv (13, 15)
poemes-verlaine.csv (12, 15)
poemes-rimbaud.csv (13, 15)
haikus-japonais.csv (47, 15)
anthologie-haikus-livres.csv (68, 15)
haikus-basho-buson-issa-shiki-santoka.csv (66, 15)
poemes-jean-de-la-fontaine.csv (15, 15)
Done.
 ---  (356, 15)
Done.


In [3]:

babe = os.listdir("data/babelio")
ldf = []
print('Concatenating babelio data...')
for file in babe:
    df = pd.read_csv(f"data/babelio/{file}")
    print(file, len(df))

Concatenating babelio data...
Christian Bobin.csv 140
Michel Foucault.csv 720
Platon.csv 893
Issa.csv 177
Aristote.csv 279
Paulo Coelho.csv 4381
Fernando Pessoa.csv 40
Christophe Andre.csv 4179
Jacques Lacan.csv 979
Spinoza.csv 704
Buson Yosa.csv 208
Shoichi Taneda.csv 87
Jack Kerouac.csv 1014
Paul Verlaine.csv 914
Alain Damasio.csv 1642
Francoise Dolto.csv 339
Basho Matsuo.csv 438
Philippe Jaccottet.csv 140
Matthieu Ricard.csv 837
Andre Breton.csv 610
Andre Comte Sponville.csv 882
ric Emmanuel Schmitt.csv 140
Albert Camus.csv 100
Shiki Masaoka.csv 103
Ito Ogawa.csv 1423
Natsume Soseki.csv 671
Paul luard.csv 1687
Rene Char.csv 1435
Rene Barjavel.csv 480
Andre Gide.csv 140
Arthur Schopenhauer.csv 788
Thomas dAnsembourg.csv 156
Rainer Maria Rilke.csv 120
Hiromi Kawakami.csv 310
Charles Bukowski.csv 2039
Marshall B Rosenberg.csv 388
Dalai Lama.csv 550
Frederic Lenoir.csv 360
Gilles Deleuze.csv 462
Michel Onfray.csv 2377
Yasunari Kawabata.csv 619
Friedrich Nietzsche.csv 3202
Boris Cyrulnik